In [53]:
### load package
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re
from sqlalchemy.orm import Session
from datetime import datetime
from database import SessionLocal

In [55]:
def make_url(complex_id):
    complex_id = complex_id
    url = f"https://new.land.naver.com/complexes/{complex_id}?ms=37.498319,127.0413807,16&a=APT:PRE:ABYG:JGC&e=RETAIL&ad=true"
    return url

In [56]:
def all_width_make(driver):
    detail_sorting_width = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[8]/td/div/div')
    all_width_str = detail_sorting_width.text
    width_list = [item.strip() for item in all_width_str.split(',')]
    try:
        driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
    except:
        pass
    return width_list

In [57]:
complex_id = 1

In [58]:
# 평수별 매매 수 등 정보 크롤링
detail_result = {}

url = make_url(complex_id)


In [59]:

# driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

# 단지정보 열기
complex_btn = driver.find_element(By.CLASS_NAME, 'complex_link')
complex_btn.click()
time.sleep(1)


In [61]:

# 평수 목록
complex_width = all_width_make(driver)
complex_width

['82㎡', '89㎡', '103㎡', '104㎡']

In [66]:

# 평수 만큼 정보 크롤링 반복
width_sales_res = {}
for idx, complex_width_ in enumerate(complex_width):
    print(complex_width_)
    tab_num = "tab" + str(idx)
    # print(tab_num)
    width_btn = driver.find_element(By.XPATH, f'//*[@id="{tab_num}"]/span')
    width_btn.click()
        
    # 매매 수
    sales_count = driver.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[5]/td/a[1]/span')
    sales_count_str = sales_count.text

    # 전세 수
    lease_count = driver.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[5]/td/a[2]/span')
    lease_count_str = lease_count.text
    
    # 월세 수
    monthly_rent_count = driver.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[5]/td/a[3]/span')
    monthly_rent_count_str = monthly_rent_count.text

    # 단기 수
    short_term_rent_count = driver.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[5]/td/a[4]/span')
    short_term_rent_count_str = short_term_rent_count.text

    width_sales_res = {
        "complex_width": complex_width_,
        "sales_count": sales_count_str,
        "lease_count": lease_count_str,
        "monthly_rent_count": monthly_rent_count_str,
        "short_term_rent_count": short_term_rent_count_str,            

    }

8


In [67]:
width_sales_res

{'complex_width': '8',
 'sales_count': '7',
 'lease_count': '0',
 'monthly_rent_count': '0',
 'short_term_rent_count': '0'}

In [68]:
complex_width

'8'